In [1]:
import time
import pandas as pd
import numpy as np

start = time.time()

with open('data/c_pairs_anon_scored.feb') as f:
    content = f.read().splitlines()
end = time.time()

end - start

12.206573009490967

In [2]:
unique_paths = set()
pairs = []
paths = []
meetings = []

In [3]:
start = time.time()
for i in xrange(len(content)):
    if i % 10000000 == 0:
        print i, ': ', time.time()-start
    (id1, id2, path, count, numMeeting) = content[i].split('|')
    pairs.append(''.join(sorted(list([id1, id2]))))
    paths.append(path)
    meetings.append(int(numMeeting))
unique_paths = set(paths)
end = time.time()
end - start

0 :  0.000283002853394
10000000 :  28.6026608944
20000000 :  57.931199789
30000000 :  86.4847548008
40000000 :  115.37856698
50000000 :  143.395458937
60000000 :  171.445757866
70000000 :  200.694644928


239.0854218006134

In [4]:
start = time.time()
pdMeetings = pd.Series(meetings, index=pairs, dtype=np.uint8)
end = time.time()
end - start

47.46397304534912

In [5]:
start = time.time()
pdPaths = pd.Series(paths, index=pairs)
end = time.time()
end - start

34.36169505119324

In [ ]:
start = time.time()
feats = pd.get_dummies(pdPaths, sparse=True)
end = time.time()
print end - start

In [7]:
start = time.time()
combinedFeats = pd.concat([feats, pdMeetings], axis=1)
end = time.time()
print end - start

191.670361996


In [8]:
combinedFeats.shape

(78633781, 170)

In [9]:
list1 = combinedFeats.columns.tolist()[:169]
list2 = ['numOfMeetings']
list1.extend(list2)

In [10]:
combinedFeats.columns = list1

In [21]:
# read February t_space data to create label vector

In [11]:
start = time.time()

with open('data/t_pairs_anon_verbs.feb.txt') as f:
    content = f.read().splitlines()
end = time.time()

end - start

30.363579988479614

In [12]:
t_unique_paths = set()
t_pairs = []
t_paths = []
t_interactions = []

In [13]:
start = time.time()
for i in xrange(len(content)):
    (id1, id2, verb, path, count, numInteraction) = content[i].split('|')
    t_pairs.append(''.join(sorted(list([id1, id2]))))
    t_paths.append(path)
    t_interactions.append(int(numInteraction))
t_unique_paths = set(t_paths)
end = time.time()
end - start

0.5357239246368408

In [14]:
ut_pairs = []
ut_paths = []

start = time.time()
for i in xrange(len(t_pairs)):
    if t_pairs[i] not in ut_pairs:
        ut_pairs.append(t_pairs[i])
        ut_paths.append(t_paths[i])
end = time.time()
print end - start

15.8201510906


In [15]:
start = time.time()
pdTPaths = pd.Series(ut_paths, index=ut_pairs)
end = time.time()
end - start

0.026149988174438477

In [16]:
start = time.time()
t_feats = pd.get_dummies(pdTPaths, sparse=True)
end = time.time()
print end - start

2.87206697464


In [17]:
t_labels = pd.Series(1, index=ut_pairs, dtype=np.uint8)

In [18]:
start = time.time()
t_combinedFeats = pd.concat([t_feats, t_labels], axis=1)
end = time.time()
print end - start

list1 = t_combinedFeats.columns.tolist()[:81]
list2 = ['label']
list1.extend(list2)
t_combinedFeats.columns = list1
t_combinedFeats = t_combinedFeats.to_dense()

0.04012799263


In [19]:
combinedFeats.shape

(78633781, 170)

In [20]:
t_combinedFeats.shape

(47931, 82)

In [22]:
# sample 10%, this hopefully will allow concat next cell
start = time.time()
reducedFeats = combinedFeats.sample(frac=0.05).to_dense()
end = time.time()
end - start

140.0306899547577

In [23]:
reducedFeats.shape

(7863378, 170)

In [74]:
# want to know how many positive examples are left?!
start = time.time()
print len(reducedFeats.index.intersection(t_combinedFeats.index))
end = time.time()
end - start

3362


5.983328104019165

In [36]:
start = time.time()
training = pd.merge(reducedFeats, t_combinedFeats, left_index=True, right_index=True)
end = time.time()
end - start

12.71766996383667

In [42]:
s1 = reducedFeats[:10].to_dense()

In [43]:
s2 = t_combinedFeats[:10].to_dense()

In [79]:
s1['IDHash'] = s1.index
s2['IDHash'] = s2.index

AttributeError: 'Index' object has no attribute 'apply'

In [78]:
s1.merge(s2)

TypeError: type object argument after * must be a sequence, not itertools.imap